The payoff of the fixed fx rate quanto digital option should be like:

$$
payoff(S, T, K) = F_0 max(S - K, 0)
$$

where S and K are denominated in foreign currency

Our goal is to find the mu and sigma for assets denominated in foreign currency under domestic risk neutral measure Qd

For the very asset denominated in dom currency, because we let the fx rate be fixed, so that the correlation risk should be included into the drift term

$$
\frac{dS_t}{S_t} = \mu^d_{S} dt + \sigma_SdW^d_S
$$

$$
\frac{dF_t}{F_t} = \mu^d_{F} dt + \sigma_FdW_F^d
$$

$$
\mu^d_{S^*} = \mu^d_{FS} = \mu^d_F + \mu^d_S + \rho \sigma_F \sigma_S
$$

$$
\mu^d_S = \mu^d_{S^*} - \mu^d_F - \rho \sigma_F \sigma_S
= r_f - q - \rho \sigma_F \sigma_S
$$

The diffusion term will not be influenced since the asset is still denominated in foreign currency

The risk-neutral probablity is adjusted w.r.t both exchange risk and correlation risk

In [1]:
import math
import numpy as np
from BlackScholes import BlackScholes
from digital_quanto_pricer import *
from Simulator import Simulator

In [2]:
spot = 106.6
strike = 110   # demoninated in the domestic currency
tau = 0.3
q = 0.
rd = 0.04
rf = 0.05
sigma_s = 0.4
sigma_f = 0.1
rho = 0.2
quanto_factor = 4

### Analytical Solution of Fixed FX Strike Quanto Digital

In [3]:
call = FixedFXStrikeDigitalQuantoCall(rd, q, rf, sigma_s, sigma_f, rho, tau, spot, strike, quanto_factor)
call.price()

1.665088941855884

### Monte Carlo Process

In [4]:
time_steps = 1500
dt = tau / time_steps
num_paths = 100000

In [5]:
mu = rf - q - rho * sigma_s * sigma_f
sigma = sigma_s

In [6]:
%%time
payoffs = Simulator.simulate_sobol(spot, mu, q, sigma, dt, num_paths, time_steps)

CPU times: user 12.1 s, sys: 6.28 s, total: 18.4 s
Wall time: 17 s


In [7]:
x = [1 if payoff[-1] > strike else 0 for payoff in payoffs]
quanto_factor * np.mean(x) * math.exp(-rf * tau)

1.6651150092722646

### Validation for put option

In [9]:
put = FixedFXStrikeDigitalQuantoPut(rd, q, rf, sigma_s, sigma_f, rho, tau, spot, strike, quanto_factor)
put.price()

2.282195142367509

In [10]:
quanto_factor * (1 - np.mean(x)) * math.exp(-rf * tau)

2.2753327491399857